In [1]:
import re
import urllib
import datetime as dt
from time import perf_counter
from pathlib import Path


import pandas as pd
import pyodbc
import sqlalchemy as sqla

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

def get_odbc_driver():
    # gets name of ODBC driver, with name "ODBC Driver <version> for SQL Server"
    drivers = [d for d in pyodbc.drivers() if 'ODBC Driver ' in d]
    
    if len(drivers) == 0:
        errmsg = f"ERROR. No usable ODBC Driver found for SQL Server." \
        f"drivers found include {drivers}. Check ODBC Administrator program" \
        "for more information."
        
        raise Exception (errmsg)
    else:
        d_versions = [re.findall('\d+', dv)[0] for dv in drivers] # [re.findall('\d+', dv)[0] for dv in drivers]
        latest_version = max([int(v) for v in d_versions])
        driver = f"ODBC Driver {latest_version} for SQL Server"
    
        return driver

driver = get_odbc_driver()

print("modules loaded.")

modules loaded.


In [2]:
# BASED ON TESTING, THIS IS A STILL SLOW BUT FASTER THAN ESRI CHUNK OF CODE TO RUN
# A SQL QUERY AND LOAD DIRECTLY INTO AN ESRI FEATURE CLASS

#-------------SPECIFY WHICH TABLES FOR DIFFERENT YEARS-----------------
ilut_tbl = 'ilut_combined2035_177_DPS'  # "ilut_combined2020_63_DPS"
eto_tbl = 'raw_eto2035_DPS_latest' # 'raw_eto2020_DPS_latest'
data_year = 2035 # 2035 or 2020

output_gdb = r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb'  # r'I:\SDE_Connections\SDE-PPA\owner@PPA.sde'  # r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb'


#-----------SPECIFY COLUMNS TO USE (SHOULD NOT VARY WITH YEAR)
jnkey_ilut = 'PARCELID'
pclpt_x = 'XCOORD'
pclpt_y = 'YCOORD'
ilut_cols = [jnkey_ilut, pclpt_x, pclpt_y, 'GISAc', 'JURIS', 'County', 'DU',
             'POP_TOT', 'HH_hh', 'ENR_K12', 'EMPTOT', 'EMPFOOD', 'EMPRET', 'EMPSVC', 'EMPIND', 
            'PT_TOT_RES', 'SOV_TOT_RES', 'HOV_TOT_RES', 'TRN_TOT_RES', 'BIK_TOT_RES', 'WLK_TOT_RES', 
             'VMT_TOT_RES']

jnkey_eto = 'PARCELID'
eto_cols = ['LU'] # exclude join key because you do not want duplicate columns

#------------OTHER SPECIFICATIONS----
assert_types = {pclpt_x: 'float', pclpt_y: 'float', 'GISAc': 'float'}
rename_dict = {'DU': 'DU_TOT', 'LU': 'LUTYPE'}


#------------BUILD QUERY--------------
ilut_cols = ', '.join(f"ilut.{fname}" for fname in ilut_cols)
eto_cols = ', '.join(f"ilut.{fname}" for fname in eto_cols)

query_str = f"""SELECT
    {ilut_cols},
    {eto_cols}
    FROM {ilut_tbl} ilut
        JOIN {eto_tbl} eto
            ON ilut.{jnkey_ilut} = eto.{jnkey_eto}"""

lutag = ilut_tbl.split('ilut_combined')[1]
out_tbl_name = f"ppa_pclpnt{lutag}"
output_tbl = str(Path(output_gdb).joinpath(f"parcel_data_pts_{data_year}"))

print("outputs specified. Ready to create table.")

outputs specified. Ready to create table.


In [3]:
#---------CREATE ITERABLE DATAFRAME WITH CHUNKS----------------
servername = 'SQL-SVR'
dbname = 'MTP2024'
trustedconn = 'yes'

conn_str = f"DRIVER={driver};" \
        f"SERVER={servername};" \
        f"DATABASE={dbname};" \
        f"Trusted_Connection={trustedconn}"

conn_str = urllib.parse.quote_plus(conn_str)
engine = sqla.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

df_itr = pd.read_sql_query(sql=query_str, con=engine, chunksize=1000)

print("df chunk iterator created.")

#------GO THROUGH CHUNKS AND LOAD INTO FEATURE CLASS-----------------
st = perf_counter()

rowcnt = 0
for i, chunk in enumerate(df_itr):
    rowcnt += chunk.shape[0]
    
    chunk = chunk.rename(columns=rename_dict)
    for fn, dtyp in assert_types.items():
        chunk[fn] = chunk[fn].astype(dtyp)
        
    chunk_s = pd.DataFrame.spatial.from_xy(chunk, x_column=pclpt_x, y_column=pclpt_y, sr=2226)
    if i == 0:
        print(f"creating feature class {output_tbl}...")
        chunk_s.spatial.to_featureclass(output_tbl, sanitize_columns=False)
        
        out_tbl_fnames = [f.name for f in arcpy.ListFields(output_tbl)]
        fields_to_use = [f for f in out_tbl_fnames if f in chunk.columns]
        fields_to_use.append('SHAPE@XY')
        print("loading rows...")
    else:
        drecs = chunk.to_dict(orient='records')
        with arcpy.da.InsertCursor(output_tbl, field_names=fields_to_use) as inscur:
            for rec in drecs:
                try:
                    coords = (rec[pclpt_x], rec[pclpt_y])
                    row = [rec[fname] for fname in out_tbl_fnames if fname in fields_to_use] # put into correct output order
                    row.append(coords)
                    inscur.insertRow(row)
                except RuntimeError:
                    vlengths = {fname: len(v) for fname, v in rec.items() if isinstance(v, str)} # length of string vals in current row
                    fclengths = {f.name: f.length for f in arcpy.ListFields(output_tbl) \
                                 if f.type == 'String'} # defined field lengths in feature class
                    try:
                        for f, fc_flen in fclengths.items():
                            rowvlen = vlengths.get(f)
                            if rowvlen and rowvlen > fc_flen: # if needed, update the fc field length to accommodate the new string val
                                arcpy.management.AlterField(output_tbl, field=f, field_length=rowvlen)
                                print(f"\tupdated field {f} to accommodate longer string value.")
                        inscur.insertRow(row)
                    except:
                        import pdb; pdb.set_trace()
                    
                    continue
                    
        if rowcnt % 100_000 == 0:
            print(f"\t{rowcnt} rows loaded...")
                
elapsed = round((perf_counter() - st) / 60, 1)
print(f"{rowcnt} rows inserted in {elapsed} mins.")
   

df chunk iterator created.
creating feature class I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\parcel_data_pts_2035...
loading rows...
	updated field JURIS to accommodate longer string value.
	100000 rows loaded...
	200000 rows loaded...
	300000 rows loaded...
	400000 rows loaded...
	500000 rows loaded...
	600000 rows loaded...
	700000 rows loaded...
	800000 rows loaded...
830280 rows inserted in 7.4 mins.


In [4]:
# SPATIAL JOIN EJ DATA WITH FILTER APPLIED

pcltbl = output_tbl
pcl_ej_field = 'EJ_AREA'
ej_layer = r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\EJ_2025_final'

fl_ej = 'fl_ej'
fl_pcl = 'fl_pcl'

for fl in [fl_ej, fl_pcl]:
    if arcpy.Exists(fl): arcpy.management.Delete(fl)

ej_fields = ['EJ_Label', 'Notes']
arcpy.management.MakeFeatureLayer(ej_layer, fl_ej)

ej_filter = "EJ_Label <> 'Minority' Or Notes = 'Equity Priority Area'"
arcpy.management.SelectLayerByAttribute(fl_ej, where_clause=ej_filter)

print("updating EJ area tags for parcels...")
if pcl_ej_field not in [f.name for f in arcpy.ListFields(pcltbl)]:
    arcpy.management.AddField(pcltbl, pcl_ej_field, field_type='SHORT')

# set default to not be EJ area (0)
with arcpy.da.UpdateCursor(pcltbl, [pcl_ej_field]) as ucur:
    for row in ucur:
        row[0] = 0 # 1 = EJ area
        ucur.updateRow(row)
        
# then for parcels within EJ areas, set EJ=1
arcpy.management.MakeFeatureLayer(pcltbl, fl_pcl)
arcpy.management.SelectLayerByLocation(fl_pcl, overlap_type='WITHIN', select_features=fl_ej)
with arcpy.da.UpdateCursor(fl_pcl, [pcl_ej_field]) as ucur:
    for row in ucur:
        row[0] = 1 # 1 = is EJ area
        ucur.updateRow(row)
        
arcpy.management.SelectLayerByAttribute(fl_pcl, selection_type='CLEAR_SELECTION')
arcpy.management.SelectLayerByAttribute(fl_ej, selection_type='CLEAR_SELECTION')

print("updated EJ tags.")

updating EJ area tags for parcels...
updated EJ tags.


In [6]:
# MAKE PARCEL POLYGON LAYER FOR PPA
arcpy.env.overwriteOutput = True
base_polys = r'Q:\2024_MTPSCS_LandUse\4_Discussion Scenario\June2024_DiscusssionScenario_Handoff\June2024_DiscussionScenario_Handoff.gdb\DS_2050_June20204'
pt_data = r'I:\SDE_Connections\SDE-PPA\owner@PPA.sde\OWNER.parcel_data_pts_2020'

# --------------create base poly layer with just geometry and parcelid
print("creating base parcel poly layer...")
poly_fc_name = Path(output_tbl).name.replace("pts", "polys")
dest_polys = str(Path(output_tbl).parent.joinpath(poly_fc_name))

include_fields = ['PARCELID', 'Shape']

fieldinfo = arcpy.FieldInfo()
for field in arcpy.ListFields(base_polys):
    fname = field.name
    if fname in include_fields:
        # addField(field_name, new_field_name, visible, split_rule)
        fieldinfo.addField(fname, fname, 'VISIBLE', 'NONE')
    else:
        fieldinfo.addField(fname, fname, 'HIDDEN', 'NONE')

fl_polys = 'fl_polys'
if arcpy.Exists(fl_polys): arcpy.Delete_management(fl_polys)
arcpy.management.MakeFeatureLayer(base_polys, fl_polys, field_info=fieldinfo)
arcpy.conversion.ExportFeatures(in_features=fl_polys, out_features=dest_polys)


# ------add appropriate fields from parcel point layer---------
pt_fields_to_add = ['LUTYPE', 'VMT_TOT_RES']
print("adding fields {pt_fields_to_add} poly layer...")

starting_poly_fields = [f.name for f in arcpy.ListFields(dest_polys)]
for f in arcpy.ListFields(pt_data):
    if f.name in pt_fields_to_add and f.name not in starting_poly_fields:
        arcpy.management.AddField(dest_polys, field_name=f.name, field_type=f.type, field_length=f.length)

# load data-to-join into dict
print("populating fields {pt_fields_to_add} poly layer...")
data_to_join = {}
pt_field_names = ['PARCELID', *pt_fields_to_add]
with arcpy.da.SearchCursor(pt_data, field_names=pt_field_names) as scur:
    for row in scur:
        data = [row[pt_field_names.index(fname)] for fname in pt_fields_to_add]
        data_to_join[row[pt_field_names.index('PARCELID')]] = data

# transfer from dict into appropirate fields of poly table
with arcpy.da.UpdateCursor(dest_polys, field_names=pt_field_names) as ucur:
    for row in ucur:
        pclid = round(row[pt_field_names.index('PARCELID')]) # need to round because some are x.9999
        datarow = data_to_join.get(pclid)
        if datarow:
            new_data = [pclid, *datarow]
            ucur.updateRow(new_data)

print(f"polygon layer created: {dest_polys}")

creating base parcel poly layer...
adding fields {pt_fields_to_add} poly layer...
populating fields {pt_fields_to_add} poly layer...
polygon layer created: I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\parcel_data_polys_2035


In [7]:
{f.name: f.type for f in arcpy.ListFields(dest_polys)}

{'OBJECTID': 'OID',
 'Shape': 'Geometry',
 'PARCELID': 'Double',
 'Shape_Length': 'Double',
 'Shape_Area': 'Double',
 'VMT_TOT_RES': 'Double',
 'LUTYPE': 'String'}

In [10]:
# copy over to PPA SDE
def copy_to_sde(in_fc, sde_path):
    arcpy.env.overwriteOutput = True
    st = perf_counter()
    final_output_tbl = str(Path(sde_path).joinpath(Path(in_fc).name))
    arcpy.conversion.ExportFeatures(in_fc, final_output_tbl)
    elapsed = round((perf_counter() - st) / 60, 1)
    print(f"Copied results over to {final_output_tbl} in {elapsed} mins.") 

sde_locn = r'I:\SDE_Connections\SDE-PPA\owner@PPA.sde'

copy_to_sde(pcltbl, sde_locn)
copy_to_sde(dest_polys, sde_locn)

Copied results over to I:\SDE_Connections\SDE-PPA\owner@PPA.sde\parcel_data_pts_2035 in 2.2 mins.
Copied results over to I:\SDE_Connections\SDE-PPA\owner@PPA.sde\parcel_data_polys_2035 in 2.8 mins.


In [9]:
dest_polys

'I:\\Projects\\Darren\\PPA3_GIS\\PPA3_GIS.gdb\\parcel_data_polys_2035'